----

![Virus Diagram](images/virusbanner.jpg)
# COVID-19 Data Retrieval Notebook 
#### Development Team: 
Equipo F (Cecilia Acosta, Luis Bonilla, Felipe Murillo)
#### Date: 
May 2020
#### Overview
This notebook is used to gather all necessary data required to analyze COVID-19 trends:
- World Health Organization (CSV and API):
     - COVID-19 Cases per coungtry
     - Hospital Beds per 10,000 ppl
- World Bank Organization (CSV and API):
     - Health Expenditures
     - Income per capita
- REST Countries (API)
     - Collect 3 character ISO country codes for country names

#### Required Inputs:
- Data_ WHO Coronavirus Covid-19 Cases and Deaths - WHO-COVID-19-global-data.csv
- Health_Expenditure_USDpercapita.csv
- HealthExpenditure_%GDP.csv
- Metadata_Country.csv

#### Outputs:
- CovidPerCountry.csv
- Health Expenditure Master.csv
- **Master COVID Research.csv**
----

### Configure Dependencies

In [1]:
# Dependencies and Setup
import os
import pandas as pd
import numpy as np
import json
import requests

----

### Data Retrieval: World Health Organization (WHO) API - Number of Hospital Beds 

In [2]:
# Create the desired URL for WHO data requests

# WHO Code corresponding to the desired metric: 
# Hospitals per 10,000 ppl
WHO_code = "WHS6_102"

# Concatenate the query URL
url = "http://apps.who.int/gho/athena/api/GHO/"+WHO_code+".json?profile=simple"

# Make JSON request to WHO API
who_data = requests.get(url).json()

In [3]:
# Peek JSON formatted data
print(json.dumps(who_data,indent=4, sort_keys=True))

{
    "dimension": [
        {
            "display": "Indicator",
            "label": "GHO"
        },
        {
            "display": "PUBLISH STATES",
            "label": "PUBLISHSTATE"
        },
        {
            "display": "Year",
            "label": "YEAR"
        },
        {
            "display": "WHO region",
            "label": "REGION"
        },
        {
            "display": "Country",
            "label": "COUNTRY"
        }
    ],
    "fact": [
        {
            "Comments": "EURO HlthRes-DB 14Sep2016",
            "Value": "29",
            "dim": {
                "COUNTRY": "Albania",
                "GHO": "Hospital beds (per 10 000 population)",
                "PUBLISHSTATE": "Published",
                "REGION": "Europe",
                "YEAR": "2012"
            }
        },
        {
            "Comments": "EURO HlthRes-DB 14Sep2016",
            "Value": "30",
            "dim": {
                "COUNTRY": "Albania",
                "GHO": "

In [4]:
# Determine how many countries are included in this data set
nCountries = len(who_data["fact"])

# Initialize lists
nameCountry = []       # name of country
regionCountry = []     # region country is in
bedsCountry = []       # hospital beds in country
dataYrCountry = []     # year data was sampled

# Start gathering data for all countries. 
# If a country's data can not be obtained then skip entry and move on to the next...

for country in np.arange(0,nCountries,1):
    
    try:
        nameCountry.append(who_data["fact"][country]["dim"]["COUNTRY"])
        regionCountry.append(who_data["fact"][country]["dim"]["REGION"])
        bedsCountry.append(float(who_data["fact"][country]["Value"]))
        dataYrCountry.append(int(who_data["fact"][country]["dim"]["YEAR"]))

    except:
        print(f"Country Not Listed in index {country}... skipping.")


Country Not Listed in index 320... skipping.


In [5]:
# Construct dataframe from with hospital data lists
raw_hospital_df = pd.DataFrame({"Country":nameCountry,
                                "Region":regionCountry,
                                "Hospital Beds":bedsCountry,
                                "Year Sampled":dataYrCountry})

In [6]:
# Create a pivot table to facilitate pulling the latest data in hospital beds for each country
grpHospital = pd.pivot_table(raw_hospital_df, values=["Hospital Beds"], index=["Country","Region"],columns=["Year Sampled"])
grpHospital["Latest Beds"] = grpHospital.ffill(axis=1).iloc[:, -1]
grpHospital.head()

Hospital Beds                    \
Year Sampled                                       2000  2001  2002  2003   
Country             Region                                                  
Afghanistan         Eastern Mediterranean           3.0   4.0   4.0   4.0   
Albania             Europe                         33.0  33.0  31.0  31.0   
Algeria             Africa                          NaN   NaN   NaN   NaN   
Angola              Africa                          NaN   NaN   NaN   NaN   
Antigua and Barbuda Americas                        NaN   NaN   NaN   NaN   

                                                                              \
Year Sampled                               2004  2005  2006  2007 2008  2009   
Country             Region                                                     
Afghanistan         Eastern Mediterranean   4.0   4.0   4.0   4.0  4.0   4.0   
Albania             Europe                 30.0  31.0  31.0  31.0  NaN  30.0   
Algeria             Africa                 17.0   NaN   NaN   NaN  NaN   NaN   
Angola              Africa                  NaN   8.0   NaN   NaN  NaN   NaN   
Antigua and Barbuda Americas                NaN   NaN   NaN   NaN  NaN   NaN   

                                                                               \
Year Sampled                               2010  2011  2012  2013  2014  2015   
Country             Region                                                      
Afghanistan         Eastern Mediterranean   4.0   4.0   5.0   5.0   5.0   5.0   
Albania             Europe                 30.0  26.0  29.0  29.0   NaN   NaN   
Algeria             Africa                  NaN   NaN   NaN   NaN   NaN  19.0   
Angola              Africa                  NaN   NaN   NaN   NaN   NaN   NaN   
Antigua and Barbuda Americas               22.0  21.0   NaN  39.0  38.0   NaN   

                                          Latest Beds  
Year Sampled                                           
Country             Region                             
Afghanistan         Eastern Mediterranean         5.0  
Albania             Europe                       29.0  
Algeria             Africa                       19.0  
Angola              Africa                        8.0  
Antigua and Barbuda Americas                     38.0

In [7]:
# Extract only the required data from the above pivot table
hospitalBedData = pd.DataFrame(grpHospital["Latest Beds"]).reset_index()
hospitalBedData.head()

,Country,Region,Latest Beds
0,Afghanistan,Eastern Mediterranean,5.0
1,Albania,Europe,29.0
2,Algeria,Africa,19.0
3,Angola,Africa,8.0
4,Antigua and Barbuda,Americas,38.0


----

### Data Retrieval: World Health Organization (WHO) CSV - COVID-19 DATA (dated: May 5, 2020)

In [8]:
# Specify CSV file containing COVID-19 data
Covid_file = os.path.join(".","inputs/Data_ WHO Coronavirus Covid-19 Cases and Deaths - WHO-COVID-19-global-data.csv")

# Import CSV file as Dataframe
Covid_df = pd.read_csv(Covid_file)
Covid_df.head()

,OBJECTID,ISO_2_CODE,ISO_3_CODE,ADM0_NAME,date_epicrv,NewCase,CumCase,NewDeath,CumDeath,Short_Name_ZH,Short_Name_FR,Short_Name_ES,Short_Name_RU,Short_Name_AR
0,1,AF,AFG,Afghanistan,2020-02-24T00:00:00.000Z,1,1,0,0,阿富汗,Afghanistan,Afganistán,Афганистан,أفغانستان
1,2,AF,AFG,Afghanistan,2020-02-25T00:00:00.000Z,0,1,0,0,阿富汗,Afghanistan,Afganistán,Афганистан,أفغانستان
2,3,AF,AFG,Afghanistan,2020-02-26T00:00:00.000Z,0,1,0,0,阿富汗,Afghanistan,Afganistán,Афганистан,أفغانستان
3,4,AF,AFG,Afghanistan,2020-02-27T00:00:00.000Z,0,1,0,0,阿富汗,Afghanistan,Afganistán,Афганистан,أفغانستان
4,5,AF,AFG,Afghanistan,2020-02-28T00:00:00.000Z,0,1,0,0,阿富汗,Afghanistan,Afganistán,Афганистан,أفغانستان


In [9]:
# Remove unnecessary columns from data frame
Covid_df = Covid_df.drop(['OBJECTID','ISO_2_CODE',
                          'Short_Name_ZH', 'Short_Name_FR',
                         'Short_Name_ES','Short_Name_RU',
                         'Short_Name_AR','NewCase',
                         'NewDeath'], axis=1)

# Rename columns to something more significant
Covid_df = Covid_df.rename(columns={"ISO_3_CODE":"Code", "ADM0_NAME":"Country",
                                   "date_epicrv":"Date"})

# Set index to "Country"
Covid_df = Covid_df.set_index("Country")

# Extract date from timestamp
Covid_df['Date'] = Covid_df['Date'].str[:10]
Covid_df.head()

,Code,Date,CumCase,CumDeath
Country,,,,
Afghanistan,AFG,2020-02-24,1,0
Afghanistan,AFG,2020-02-25,1,0
Afghanistan,AFG,2020-02-26,1,0
Afghanistan,AFG,2020-02-27,1,0
Afghanistan,AFG,2020-02-28,1,0


In [10]:
# Create new dataframe with the latest # of cases & deaths 
CovidMaxDeath = Covid_df.groupby(["Country","Code"])[["CumDeath"]].max()
CovidMaxCases = Covid_df.groupby(["Country","Code"])[["CumCase"]].max()
CovidPerCountry = pd.concat([CovidMaxCases,CovidMaxDeath], axis=1).reset_index()
CovidPerCountry = CovidPerCountry.set_index("Country")

# Export dataframe as a CSV
CovidPerCountry.to_csv(os.path.join(".","outputs/CovidPerCountry.csv"),index=True)
CovidPerCountry.head()

,Code,CumCase,CumDeath
Country,,,
Afghanistan,AFG,3224,95
Albania,ALB,803,31
Algeria,DZA,4648,465
Andorra,AND,751,45
Angola,AGO,35,2


----

### Data Retrieval: World Bank - Health Expenditures by Country (CSV)

In [11]:
# Import CSV file containing "health expenditure data as percentage of GDP" into a dataframe
health_expp = pd.read_csv(os.path.join(".","inputs/HealthExpenditure_%GDP.csv"), encoding='cp1252')
health_expp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,NaN,NaN,9.443390,8.941258,9.808473,9.948289,...,8.569671,8.561907,7.897176,8.805940,9.528870,10.105348,10.961983,11.777194,NaN,NaN
2,Angola,AGO,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,1.908599,4.483516,3.329461,3.547973,3.967201,2.852197,...,2.695100,2.645608,2.395750,2.732828,2.434129,2.605795,2.713149,2.791500,NaN,NaN
3,Albania,ALB,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,9.348101,9.363031,9.119281,8.463996,8.210544,8.565982,...,9.449671,9.765894,10.046941,10.112114,10.276479,10.252713,10.324107,10.320552,NaN,NaN


In [12]:
# Create a new dataframe by dropping unnecessary columns from imported dataframe
health_expp2=health_expp.drop(columns=['Indicator Name', 'Indicator Code'])

# Set dataframe index to country and country code
health_expp2=health_expp2.set_index(["Country Name","Country Code"])

# Obtain the lastest data in health expenditures
health_expp2["Latest%"]=health_expp2.ffill(axis=1).iloc[:, -1]

health_expp2.head()

,,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Latest%
Country Name,Country Code,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,AFG,NaN,NaN,9.443390,8.941258,9.808473,9.948289,10.622766,9.904674,10.256495,9.818488,...,8.561907,7.897176,8.805940,9.528870,10.105348,10.961983,11.777194,NaN,NaN,11.777194
Angola,AGO,1.908599,4.483516,3.329461,3.547973,3.967201,2.852197,2.685537,2.974388,3.322905,3.842609,...,2.645608,2.395750,2.732828,2.434129,2.605795,2.713149,2.791500,NaN,NaN,2.791500
Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,AND,9.348101,9.363031,9.119281,8.463996,8.210544,8.565982,8.434281,8.428402,8.792542,9.026737,...,9.765894,10.046941,10.112114,10.276479,10.252713,10.324107,10.320552,NaN,NaN,10.320552


In [13]:
# Create a new dataframe containing only the latest data
health_expp3=health_expp2.reset_index()
health_expp3=health_expp3[["Country Name","Country Code","Latest%"]]
health_expp3.head()

,Country Name,Country Code,Latest%
0,Aruba,ABW,NaN
1,Afghanistan,AFG,11.777194
2,Angola,AGO,2.791500
3,Albania,ALB,NaN
4,Andorra,AND,10.320552


In [14]:
# Read in CSV file with "health expenditure data in $USD per capita"
health_expa = pd.read_csv(os.path.join(".","inputs/Health_Expenditure_USDpercapita.csv"), encoding='cp1252')
health_expa.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,16.249540,17.490738,20.927086,24.446512,...,45.587749,51.553257,52.218510,55.967548,60.112759,60.088814,61.451286,67.122650,NaN,NaN
2,Angola,AGO,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,12.963032,28.854246,28.961367,34.718300,49.526726,53.930702,...,96.643700,122.117798,122.242943,143.703217,131.751877,108.680672,95.220802,114.459641,NaN,NaN
3,Albania,ALB,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,2050.647461,2081.275391,2256.349121,2774.089600,3161.482178,3536.122803,...,3754.731445,4013.911865,3857.161133,4107.733887,4346.258301,3695.067871,3843.702881,4040.786621,NaN,NaN


In [15]:
# Create a new dataframe by dropping unnecessary columns from imported dataframe
health_expa2=health_expa.drop(columns=['Indicator Name', 'Indicator Code'])

# Set dataframe index to country and country code
health_expa2=health_expa2.set_index(["Country Name","Country Code"])

# Obtain the lastest data in health expenditures
health_expa2["Latest$"]=health_expa2.ffill(axis=1).iloc[:, -1]
health_expa2

,,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Latest$
Country Name,Country Code,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,AFG,NaN,NaN,16.249540,17.490738,20.927086,24.446512,28.416662,31.840185,38.700493,42.304508,...,51.553257,52.218510,55.967548,60.112759,60.088814,61.451286,67.122650,NaN,NaN,67.122650
Angola,AGO,12.963032,28.854246,28.961367,34.718300,49.526726,53.930702,69.424698,92.452011,135.208298,119.808624,...,122.117798,122.242943,143.703217,131.751877,108.680672,95.220802,114.459641,NaN,NaN,114.459641
Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,AND,2050.647461,2081.275391,2256.349121,2774.089600,3161.482178,3536.122803,3689.705566,4094.544434,4201.729492,3911.895996,...,4013.911865,3857.161133,4107.733887,4346.258301,3695.067871,3843.702881,4040.786621,NaN,NaN,4040.786621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Yemen, Rep.",YEM,25.154909,26.497791,27.313015,34.842796,38.219463,42.373791,49.861038,57.953300,69.357399,65.477837,...,64.361031,73.826073,78.390152,79.746155,72.039467,NaN,NaN,NaN,NaN,72.039467
South Africa,ZAF,221.782974,195.196320,169.022186,254.303711,322.716400,354.062439,362.463623,386.000031,372.355102,413.074097,...,597.359436,579.748535,526.499573,509.833221,470.796967,428.296875,499.237549,NaN,NaN,499.237549


In [16]:
# Create a new dataframe containing only the latest data
health_expa3=health_expa2.reset_index()
health_expa3=health_expa3[["Country Name","Country Code","Latest$"]]
health_expa3.head()

,Country Name,Country Code,Latest$
0,Aruba,ABW,NaN
1,Afghanistan,AFG,67.122650
2,Angola,AGO,114.459641
3,Albania,ALB,NaN
4,Andorra,AND,4040.786621


In [17]:
# Import CSV file grouping countries into income groups
catalogue = pd.read_csv(os.path.join(".","inputs/Metadata_Country.csv"),encoding='cp1252')
catalogue.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFG,South Asia,Low income,NaN,Afghanistan,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,ALB,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,AND,Europe & Central Asia,High income,NaN,Andorra,NaN


In [18]:
# Extract columns of interest
health_exp=catalogue[["TableName","Country Code","Region","IncomeGroup"]]

# Rename column to something more significant
health_exp=health_exp.rename(columns={"TableName": "Country Name"})

health_exp.head()

,Country Name,Country Code,Region,IncomeGroup
0,Aruba,ABW,Latin America & Caribbean,High income
1,Afghanistan,AFG,South Asia,Low income
2,Angola,AGO,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,Europe & Central Asia,Upper middle income
4,Andorra,AND,Europe & Central Asia,High income


In [19]:
# Merge income group data into "health expenditure data in $USD per capita" dataframe
health_expfull=pd.merge(health_exp, health_expa3, on="Country Name", how="left")

health_expfull.head()

,Country Name,Country Code_x,Region,IncomeGroup,Country Code_y,Latest$
0,Aruba,ABW,Latin America & Caribbean,High income,ABW,NaN
1,Afghanistan,AFG,South Asia,Low income,AFG,67.122650
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,AGO,114.459641
3,Albania,ALB,Europe & Central Asia,Upper middle income,ALB,NaN
4,Andorra,AND,Europe & Central Asia,High income,AND,4040.786621


In [20]:
# Merge "health expenditure data as percentage of GDP" to create complete helath expenditure dataframe
health_expfull=pd.merge(health_expfull, health_expp3, on="Country Name", how="left")
health_expfull.head()

,Country Name,Country Code_x,Region,IncomeGroup,Country Code_y,Latest$,Country Code,Latest%
0,Aruba,ABW,Latin America & Caribbean,High income,ABW,NaN,ABW,NaN
1,Afghanistan,AFG,South Asia,Low income,AFG,67.122650,AFG,11.777194
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,AGO,114.459641,AGO,2.791500
3,Albania,ALB,Europe & Central Asia,Upper middle income,ALB,NaN,ALB,NaN
4,Andorra,AND,Europe & Central Asia,High income,AND,4040.786621,AND,10.320552


In [21]:
# Remove duplicated columns
health_expfull=health_expfull.drop(columns=["Country Code_y","Country Code"])

# Rename columns
health_expfull=health_expfull.rename(
    columns={"Country Name": "Country",
            "Country Code_x": "Code",
            "Latest$":"Health Expenditure per capita (current USD)",
            "Latest%":"Health Expenditure (% of GDP)"
            })
health_expfull

,Country,Code,Region,IncomeGroup,Health Expenditure per capita (current USD),Health Expenditure (% of GDP)
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,67.122650,11.777194
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,114.459641,2.791500
3,Albania,ALB,Europe & Central Asia,Upper middle income,NaN,NaN
4,Andorra,AND,Europe & Central Asia,High income,4040.786621,10.320552
...,...,...,...,...,...,...
258,Kosovo,XKX,Europe & Central Asia,Upper middle income,NaN,NaN
259,"Yemen, Rep.",YEM,Middle East & North Africa,Low income,72.039467,4.234575
260,South Africa,ZAF,Sub-Saharan Africa,Upper middle income,499.237549,8.113118
261,Zambia,ZMB,Sub-Saharan Africa,Lower middle income,67.648666,4.470341


In [22]:
# Export master dataframe containing all health expenditures as a CSV for future use
health_expfull.to_csv(os.path.join(".","outputs/Health Expenditure Master.csv"))

----

### Pre-Data Retrieval API: Obtaining ISO codes for Countries

In [23]:
# Before making country specific requests, need a mapping of country name to iso2code (used by World Bank to make
# individual country requests)

# World Bank iso3code listing for all countries
code_url = "https://restcountries.eu/rest/v2/all"

# Make JSON request to Rest Countries API
country_data = requests.get(code_url).json()

# Peek JSON formatted data
print(json.dumps(country_data,indent=4, sort_keys=True))

[
    {
        "alpha2Code": "AF",
        "alpha3Code": "AFG",
        "altSpellings": [
            "AF",
            "Af\u0121\u0101nist\u0101n"
        ],
        "area": 652230.0,
        "borders": [
            "IRN",
            "PAK",
            "TKM",
            "UZB",
            "TJK",
            "CHN"
        ],
        "callingCodes": [
            "93"
        ],
        "capital": "Kabul",
        "cioc": "AFG",
        "currencies": [
            {
                "code": "AFN",
                "name": "Afghan afghani",
                "symbol": "\u060b"
            }
        ],
        "demonym": "Afghan",
        "flag": "https://restcountries.eu/data/afg.svg",
        "gini": 27.8,
        "languages": [
            {
                "iso639_1": "ps",
                "iso639_2": "pus",
                "name": "Pashto",
                "nativeName": "\u067e\u069a\u062a\u0648"
            },
            {
                "iso639_1": "uz",
                "iso639_2

In [24]:
# Determine how many countries are included in this data set
nCountriesCodes = len(country_data)

# Initialize lists
nameCountry = []       # name of country
a3cCountry = []        # ISO 3166-1 3-letter country code
latlng = []            # geocache data

# Start gathering data for all countries. 
# If a country's data can not be obtained then skip entry and move on to the next...

for code in np.arange(0,nCountriesCodes,1):
    
    try:
        nameCountry.append(country_data[code]["name"])
        a3cCountry.append(country_data[code]["alpha3Code"])
        latlng.append(country_data[code]["latlng"])

    except:
        print(f"Country Not Listed in index {code}... skipping.")

In [25]:
# Construct dataframe from with hospital data lists
raw_codes_df = pd.DataFrame({"Country":nameCountry,
                                "Code":a3cCountry,
                                "Location":latlng})
raw_codes_df.head()

,Country,Code,Location
0,Afghanistan,AFG,"[33.0, 65.0]"
1,Åland Islands,ALA,"[60.116667, 19.9]"
2,Albania,ALB,"[41.0, 20.0]"
3,Algeria,DZA,"[28.0, 3.0]"
4,American Samoa,ASM,"[-14.33333333, -170.0]"


### Add country codes to hospital bed dataframe

In [26]:
# Add country code and location data to hospital bed dataframe
total_df = hospitalBedData.merge(raw_codes_df,left_on= "Country",right_on="Country",how="left")

# The following countries were not merged correctly (i.e., country names differ in spelling)
unfoundCountries = total_df.loc[pd.isnull(total_df["Code"])]
print(f"List of unmatched coutries:\n{unfoundCountries['Country']}")

List of unmatched coutries:
43                                   Czechia
45     Democratic People's Republic of Korea
46          Democratic Republic of the Congo
56                                  Eswatini
132                        Republic of Korea
133                      Republic of Moldova
134              Republic of North Macedonia
172              United Republic of Tanzania
Name: Country, dtype: object


In [27]:
# Rather than remove unmerged countries, manual corrections will be made to not jeopardize the data set
corrections = {"Czechia":"Czech Republic",
              "Democratic People's Republic of Korea":"Korea (Democratic People's Republic of)",
              "Republic of Korea":"Korea (Republic of)",
              "Democratic Republic of the Congo":"Congo (Democratic Republic of the)",
              "Eswatini":"Swaziland",
              "Republic of Moldova":"Moldova (Republic of)",
              "Republic of North Macedonia":"Macedonia (the former Yugoslav Republic of)",
              "United Republic of Tanzania":"Tanzania, United Republic of"}

# Make name corrections into hospital bed data set
for country in corrections:
    oldName = country
    newName = corrections[country]
    hospitalBedData.loc[hospitalBedData["Country"] == oldName,"Country"] = newName

In [28]:
# Merge again...
total_df = hospitalBedData.merge(raw_codes_df,left_on= "Country",right_on="Country",how="left")

# We should have a complete merge!
unfoundCountries = total_df.loc[pd.isnull(total_df["Code"])]
print(f"List of unmatched coutries:\n {unfoundCountries}")
    

List of unmatched coutries:
 Empty DataFrame
Columns: [Country, Region, Latest Beds, Code, Location]
Index: []


### Data Retrieval: World Bank API - Income per capita

In [29]:
# Create the desired URL for World Bank data requests

# World Bank Organization corresponding to desired indicator: 
# Gross National Income, converted to U.S. dollars 
WB_searchCode = "NY.GNP.PCAP.CD"

yearSpan = 10      # going back this # of years from latest year
latestYr = 2019    # data available for most recent year

# Initialize lists
WBcountry = []
years = np.arange(latestYr,latestYr-yearSpan,-1)
yearlyData = []
yearlyData_i = []

# For every country with hospital bed data, attempt to retrieve income data
for country in list(total_df["Code"]):

    # Concatenate the query URL
    url = "http://api.worldbank.org/v2/country/"+country+"/indicator/"+WB_searchCode+"?format=json"
    
    # Make JSON request to World Bank API
    bank_data = requests.get(url).json()
    
    # Record data: country id, yearly data for income per capita for ther past yearSpan years
    try:
        WBcountry.append(bank_data[1][0]["countryiso3code"])
        
        for yr in np.arange(0,yearSpan,1):
            yearlyData_i.append(bank_data[1][yr]["value"])
            
        yearlyData.append(yearlyData_i)
        yearlyData_i = []
        
    except:
        print(f"Income data for country {country} not found... skipping.")

Income data for country COK not found... skipping.
Income data for country NIU not found... skipping.


In [30]:
# Create data frame with income per capita data
raw_income_df = pd.DataFrame(columns=years, data=yearlyData)
raw_income_df.insert(0, 'Code', WBcountry)

# Set country code as index
raw_income_df = raw_income_df.set_index("Code")

# Record the lastest non-NaN value obtained 
raw_income_df["Latest Data"] = raw_income_df.ffill(axis=1).iloc[:, -1]

# Drop all countries with missing income data
income_data_df = raw_income_df.dropna(subset=["Latest Data"])["Latest Data"]

# Print out a summary of country data obtained
print(f"Only financial data for {len(income_data_df.index)} countries was retrieved from the original {len(total_df.index)}")

Only financial data for 176 countries was retrieved from the original 181


----

### Merge data into ultimate dataframe

In [31]:
# Add income data to hospital bed data set
bed_income_data = total_df.merge(income_data_df,left_on="Code",right_on="Code",how="right")

# Rearrange columns for visualization purposes
columns_titles = ["Country","Code","Region","Location","Latest Beds","Latest Data"]
bed_income_data=bed_income_data.reindex(columns=columns_titles)

# Rename columns for better readability
bed_income_data.rename(columns={'Latest Beds':'Hospital Beds','Latest Data':'GNI Per Capita'}, inplace=True)

# Set index to "Country" name
bed_income_data = bed_income_data.set_index("Country")

# Convert GNI per capita into numeric format
bed_income_data["GNI Per Capita"] = pd.to_numeric(bed_income_data["GNI Per Capita"], downcast="float")

bed_income_data.head()

,Code,Region,Location,Hospital Beds,GNI Per Capita
Country,,,,,
Afghanistan,AFG,Eastern Mediterranean,"[33.0, 65.0]",5.0,510.0
Albania,ALB,Europe,"[41.0, 20.0]",29.0,4360.0
Algeria,DZA,Africa,"[28.0, 3.0]",19.0,4480.0
Angola,AGO,Africa,"[-12.5, 18.5]",8.0,3240.0
Antigua and Barbuda,ATG,Americas,"[17.05, -61.8]",38.0,12690.0


In [32]:
# Merge COVID-19 data into hospital bed, income per capita data frame
FullData = bed_income_data.merge(CovidPerCountry,left_on="Code",right_on="Code",how="left")

# Provide list of unmatched countries
unfoundCountries2 = FullData.loc[pd.isnull(FullData["CumCase"])]
print(f"List of unmatched coutries:\n{unfoundCountries2['Code']}")
print("\x1b[1;31m"+"OK - These countries do not have COVID-19 data available"+'\x1b[0m')

List of unmatched coutries:
86     KIR
92     LSO
112    NRU
146    SLB
161    TKM
162    TUV
Name: Code, dtype: object
OK - These countries do not have COVID-19 data available


In [33]:
# Remove countries without COVID-19 data and reset_index
FullData2 = FullData.dropna()

FullData2

,Code,Region,Location,Hospital Beds,GNI Per Capita,CumCase,CumDeath
0,AFG,Eastern Mediterranean,"[33.0, 65.0]",5.0,510.0,3224.0,95.0
1,ALB,Europe,"[41.0, 20.0]",29.0,4360.0,803.0,31.0
2,DZA,Africa,"[28.0, 3.0]",19.0,4480.0,4648.0,465.0
3,AGO,Africa,"[-12.5, 18.5]",8.0,3240.0,35.0,2.0
4,ATG,Americas,"[17.05, -61.8]",38.0,12690.0,25.0,3.0
...,...,...,...,...,...,...,...
171,VEN,Americas,"[8.0, -66.0]",8.0,11810.0,357.0,10.0
172,VNM,Western Pacific,"[16.16666666, 107.83333333]",26.0,1250.0,271.0,0.0
173,YEM,Eastern Mediterranean,"[15.0, 48.0]",7.0,1200.0,21.0,3.0
174,ZMB,Africa,"[-15.0, 30.0]",20.0,1340.0,137.0,3.0


In [34]:
# Merge health expenditure data to summary data frame
master= FullData2.merge(health_expfull,left_on="Code",right_on="Code", how="left")

# Drop repeated column data and set country name as index
master=master.drop(columns=["Region_y"]).set_index("Country")

master.head()

,Code,Region_x,Location,Hospital Beds,GNI Per Capita,CumCase,CumDeath,IncomeGroup,Health Expenditure per capita (current USD),Health Expenditure (% of GDP)
Country,,,,,,,,,,
Afghanistan,AFG,Eastern Mediterranean,"[33.0, 65.0]",5.0,510.0,3224.0,95.0,Low income,67.122650,11.777194
Albania,ALB,Europe,"[41.0, 20.0]",29.0,4360.0,803.0,31.0,Upper middle income,NaN,NaN
Algeria,DZA,Africa,"[28.0, 3.0]",19.0,4480.0,4648.0,465.0,Upper middle income,258.494293,6.374322
Angola,AGO,Africa,"[-12.5, 18.5]",8.0,3240.0,35.0,2.0,Lower middle income,114.459641,2.791500
Antigua and Barbuda,ATG,Americas,"[17.05, -61.8]",38.0,12690.0,25.0,3.0,High income,673.859680,4.533532


In [35]:
# Provide list of unmatched countries
unfoundCountries3 = master.loc[pd.isnull(master["IncomeGroup"])]
print(f"List of unmatched coutries:\n{unfoundCountries3.index}")

List of unmatched coutries:
Index([], dtype='object', name='Country')


----

### Export Master Dataframe

In [36]:
master.to_csv(os.path.join(".","outputs/Master COVID Research.csv"))